In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.datasets import make_classification
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline


In [2]:
from google.colab import files
uploaded = files.upload()

import io
wine_df = pd.read_csv(io.BytesIO(uploaded['preprocessed_wine_data.csv']))

Saving preprocessed_wine_data.csv to preprocessed_wine_data.csv


In [3]:
X = wine_df.copy()
X = X.drop("quality", axis=1)
y = wine_df["quality"].values
dummy_y = np_utils.to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, random_state=78)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1497, 11)
(499, 11)
(1497, 10)
(499, 10)


In [4]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [5]:
clf = RandomForestClassifier(random_state=1, n_estimators=50).fit(X_train_scaled, y_train)
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

Training Score: 0.9986639946559787
Testing Score: 0.4649298597194389


In [6]:
feature_importances = clf.feature_importances_

In [7]:
feature_importances

array([0.07512825, 0.09894902, 0.07676237, 0.08145294, 0.08468685,
       0.08882275, 0.09197027, 0.10908396, 0.0965438 , 0.0779453 ,
       0.1186545 ])

In [8]:
from sklearn.feature_selection import SelectFromModel
sel = SelectFromModel(clf)
sel.fit(X_train_scaled, y_train)

SelectFromModel(estimator=RandomForestClassifier(n_estimators=50,
                                                 random_state=1))

In [9]:
X_selected_train, X_selected_test, y_train, y_test = train_test_split(sel.transform(X), dummy_y, random_state=1)
scaler = StandardScaler().fit(X_selected_train)
X_selected_train_scaled = scaler.transform(X_selected_train)
X_selected_test_scaled = scaler.transform(X_selected_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [10]:
clf2 = RandomForestClassifier(random_state=1, n_estimators=50).fit(X_selected_train_scaled, y_train)
print(f'Training Score: {clf2.score(X_selected_train_scaled, y_train)}')
print(f'Testing Score: {clf2.score(X_selected_test_scaled, y_test)}')

Training Score: 0.9986639946559787
Testing Score: 0.46292585170340683


In [11]:
print(X_selected_train_scaled.shape)
print(X_selected_test_scaled.shape)
print(y_train.shape)
print(y_test.shape)

(1497, 5)
(499, 5)
(1497, 10)
(499, 10)


In [12]:
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=5, activation='relu'))
	model.add(Dense(10, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model


In [13]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X_selected_train_scaled, y_train, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

<ipython-input-13-74202cdaa8c0>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)


Baseline: 53.77% (3.96%)


In [17]:
import tensorflow as tf


nn = tf.keras.models.Sequential()
# number_input_features = len(X_train[0])

nn.add(tf.keras.layers.Dense(units=50, activation="relu", input_dim=5))
# nn.add(tf.keras.layers.Dense(units=22, activation="relu"))
nn.add(tf.keras.layers.Dense(units=10, activation="softmax"))
nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn.fit(X_selected_train_scaled, y_train, epochs=100)
model_loss, model_accuracy = nn.evaluate(X_selected_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


Epoch 1/100
47/47 [==============================] - 1s 4ms/step - loss: 2.0710 - accuracy: 0.3380
Epoch 2/100
47/47 [==============================] - 0s 3ms/step - loss: 1.6231 - accuracy: 0.4582
Epoch 3/100
47/47 [==============================] - 0s 5ms/step - loss: 1.3887 - accuracy: 0.4923
Epoch 4/100
47/47 [==============================] - 0s 4ms/step - loss: 1.2770 - accuracy: 0.5070
Epoch 5/100
47/47 [==============================] - 0s 4ms/step - loss: 1.2185 - accuracy: 0.5144
Epoch 6/100
47/47 [==============================] - 0s 6ms/step - loss: 1.1826 - accuracy: 0.5197
Epoch 7/100
47/47 [==============================] - 0s 4ms/step - loss: 1.1575 - accuracy: 0.5257
Epoch 8/100
47/47 [==============================] - 0s 4ms/step - loss: 1.1374 - accuracy: 0.5284
Epoch 9/100
47/47 [==============================] - 0s 4ms/step - loss: 1.1226 - accuracy: 0.5344
Epoch 10/100
47/47 [==============================] - 0s 4ms/step - loss: 1.1101 - accuracy: 0.5458
Epoch 11/